# Import Libraries

In [1]:
import pandas as pd

# Obtain Data

Read csv file with medal counts into Pandas

In [2]:
medals_df = pd.read_csv('Data/medals.csv')
medals_df.head()

,COUNTRY,GOLD,SILVER,BRONZE,TOTALS
0,United States,39,41,33,113
1,China,38,32,18,88
2,Japan,27,14,17,58
3,Great Britain,22,21,22,65
4,Russian Olympic Committee,20,28,23,71


Read csv file with country indicator data into pandas

In [3]:
indicators_df = pd.read_csv('Data/indicators.csv')
indicators_df.head()

,country_name,Current health expenditure (% of GDP),"GDP per capita, PPP (constant 2017 international $)",Gini index (World Bank estimate),"Government expenditure on education, total (% of GDP)",International migrant stock (% of population),"PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)","Population ages 20-24, female (% of female population)","Population ages 20-24, male (% of male population)","Population ages 25-29, female (% of female population)","Population ages 25-29, male (% of male population)","Population ages 30-34, female (% of female population)","Population ages 30-34, male (% of male population)","Population, total",Urban population (% of total population)
0,Afghanistan,9.395727,1978.961579,NaN,3.21378,1.175547,56.910808,10.073862,10.098379,7.907260,8.063549,6.221683,6.503323,38928341.0,26.026000
1,Africa Eastern and Southern,5.925882,3371.814704,NaN,4.71930,1.734768,35.512988,9.218579,9.355607,7.906863,7.947182,6.763558,6.723442,677243299.0,36.783306
2,Africa Western and Central,4.004790,3995.039927,NaN,3.03285,2.079691,58.064482,8.940792,9.041989,7.522335,7.578698,6.425444,6.427822,458803476.0,47.848625
3,Albania,5.262714,13295.410885,33.2,3.94576,1.989036,18.200603,7.853874,8.013442,8.138641,8.828823,7.168070,8.372488,2837743.0,62.112000
4,Algeria,6.218427,10681.679297,27.6,6.10036,0.611072,38.884011,6.769581,6.910889,8.120237,8.179230,8.527349,8.474931,43851043.0,73.733000


# Scrub Data

Drop uneeded columns from indicators_df

In [6]:
df = pd.merge(medals_df, indicators_df, left_on = 'COUNTRY', right_on = 'country_name')
print(df.shape)
df.head(50)

(80, 20)


,COUNTRY,GOLD,SILVER,BRONZE,TOTALS,country_name,Current health expenditure (% of GDP),"GDP per capita, PPP (constant 2017 international $)",Gini index (World Bank estimate),"Government expenditure on education, total (% of GDP)",International migrant stock (% of population),"PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)","Population ages 20-24, female (% of female population)","Population ages 20-24, male (% of male population)","Population ages 25-29, female (% of female population)","Population ages 25-29, male (% of male population)","Population ages 30-34, female (% of female population)","Population ages 30-34, male (% of male population)","Population, total",Urban population (% of total population)
0,United States,39,41,33,113,United States,16.885298,60162.909040,41.4,NaN,14.490654,7.409442,6.540027,6.913125,6.991455,7.414830,6.787581,7.145002,3.294841e+08,82.664
1,China,38,32,18,88,China,5.351080,16410.797797,38.5,3.54249,0.071076,52.664596,5.831647,6.268071,6.627831,6.979076,8.885728,9.000131,1.402112e+09,61.428
2,Japan,27,14,17,58,Japan,10.953388,39200.657167,32.9,NaN,1.614775,11.704778,4.513509,4.980451,4.633396,5.103483,5.120588,5.628105,1.258360e+08,91.782
3,Australia,17,7,22,46,Australia,9.284598,48697.837028,34.4,5.10789,28.218410,8.550324,6.119290,6.388266,6.805434,7.163807,7.310814,7.399108,2.568704e+07,86.241
4,Netherlands,10,12,14,36,Netherlands,9.974586,54325.508829,28.1,5.35765,11.695683,12.034408,5.796915,6.118486,6.144205,6.424525,6.182796,6.390403,1.744114e+07,92.236
5,France,10,12,11,33,France,11.257895,42313.193109,32.4,5.41317,12.088479,11.814964,5.427057,5.916428,5.497921,5.768026,5.965934,6.115854,6.739158e+07,80.975
6,Germany,10,11,16,37,Germany,11.429951,51259.239556,31.9,4.99274,14.879051,12.028767,5.124005,5.752610,5.433641,6.088959,6.181597,6.815844,8.324052e+07,77.453
7,Italy,10,10,20,39,Italy,8.667634,38992.148381,35.9,4.25572,9.680768,16.750712,4.563964,5.185785,4.950022,5.520909,5.358853,5.785012,5.955402e+07,71.039
8,Canada,7,6,11,24,Canada,10.790469,45910.174955,33.3,5.26205,21.801664,6.428383,6.076998,6.450040,7.067898,7.417936,6.893269,7.025109,3.800524e+07,81.562
9,Brazil,7,6,8,21,Brazil,9.514387,14059.369598,53.4,6.08851,0.343313,12.706629,7.830381,8.316649,7.786078,8.171533,7.931868,8.207612,2.125594e+08,87.073
